1. Data Loader
2. Merge CSV into one master
3. Remove to return the cheapest flight??
3. Feature Engineering

In [5]:
import pandas as pd
import numpy as np
import zipfile 
import io
import os
import concurrent.futures
import time
import subprocess
import multiprocessing
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter


# Data Loader

## Function to unzip folder

In [5]:
# Extract the zip and save in interim
data_folder = '../data'
zip_flight_data= data_folder + '/external/itineraries_csv.zip'

extract_directory = data_folder +'/interim'

# Read the zip file
with zipfile.ZipFile(zip_flight_data, 'r') as z:
    z.extractall(extract_directory)

print(f"Extracted {zip_flight_data} to {extract_directory}")



Extracted ../data/external/itineraries_csv.zip to ../data/interim


## Function to unzip CSV and save them as 1 per airport

In [8]:
# CHAMPION FILE EXTRACTOR
def process_folder(folder):    
    df_list = []
    files = os.listdir(f"../data/interim/itineraries_csv/{folder}")
    for file in files:
        file_path = f"../data/interim/itineraries_csv/{folder}/{file}"
        if file.endswith('.zip'):
            df=pd.read_csv(file_path, compression='zip')
            df_list.append(df)
    df_list = pd.concat(df_list)
    df_list.to_csv(f"../data/interim/{folder}.csv", index=False)
    print(f"Saved {folder}.csv")
root_dir = "../data/interim/itineraries_csv"
folders = [entry.name for entry in os.scandir(root_dir) if entry.is_dir()]

# Measure execution time
start_time = time.time()

# Limit the number of threads
max_threads = 4

# Use multiprocessing.Pool to run the folder processing function in parallel
with multiprocessing.Pool(processes=max_threads) as pool:
    pool.map(process_folder, folders)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")

Saved IAD.csv
Saved JFK.csv
Saved PHL.csv
Saved ORD.csv
Saved OAK.csv
Saved DTW.csv
Saved DEN.csv
Saved LAX.csv
Saved CLT.csv
Saved EWR.csv
Saved MIA.csv
Saved BOS.csv
Saved LGA.csv
Saved ATL.csv
Saved SFO.csv
Saved DFW.csv
Execution time: 56.55 seconds


In [9]:
# Remove the processed folders to save storage space
subprocess.run(f"rm -rf ../data/interim/itineraries_csv", shell=True)
subprocess.run(f"rm -rf ../data/interim/__MACOSX", shell=True)
print("Removed folders")


Removed folders


# Read in airport CSV

In [17]:
df = pd.read_csv('../data/interim/ATL.csv')


# Transform

## Clean Data

In [19]:
# Function to clean and convert the data
def transform_data(df):

    # Convert date columns
    date_columns = ['searchDate', 'flightDate']
    df[date_columns] = df[date_columns].apply(pd.to_datetime)

    # Convert float to numeric
    float_columns = ['totalFare','totalTravelDistance']
    for col in float_columns:
        df[col] = pd.to_numeric(df[col],errors='coerce').astype('float32')

    # Drop rows
    df = df.dropna(subset=['segmentsEquipmentDescription','segmentsAirlineName'])
    return df

    


## Feature Engineering

### Segments

In [ ]:
# Covert the segment data into new variables

airports= ['ATL','BOS','DEN','DFW','EWR','JFK','LAX','LGA','MIA','OAK','ORD','PHL','SFO']
# dictionary for the correct time offsets
time_offsets = {'ATL': '-04:00', 'BOS' : '-04:00' , 'DEN':'-06:00', 'DFW':'-05:00' ,'EWR':'-04:00',   'JFK': '-04:00','LAX': '-07:00', 'LGA': '-04:00', 'MIA': '-04:00', 'OAK': '-07:00', 'ORD': '-05:00', 'PHL': '-04:00', 'SFO': '-07:00'}

# Departure Hour Processing
def process_segments_DepartTime(df):
    # Split the departure time
    print("Shape before processing: ", df.shape)
    df["segmentsDepartureTimeRaw_hour"] = df["segmentsDepartureTimeRaw"].str.split("|").str[0]
    airportcode = df["startingAirport"][0]
    correct_time_offset = time_offsets[airportcode]

    # filter for departure time hour that has the correct time offset
    df = df[df["segmentsDepartureTimeRaw_hour"].str[-6:] == correct_time_offset]
    print("Shape after processing departure time: ", df.shape)

    # check date difference between the  segmentsDepartureTimeRaw_hour aNd flight date and eliminate any that dont match
    df["segmentsDepartureTimeRaw_hour"] = pd.to_datetime(
        df["segmentsDepartureTimeRaw_hour"]
    ).dt.tz_localize(None)
    df = df[(df["segmentsDepartureTimeRaw_hour"] - df["flightDate"]).dt.days == 0]
    print("Shape after filtering segment departure not the same as flight date: ", df.shape)

    # extract out the hour of the day
    df["segmentsDepartureTimeRaw_hour"] = df["segmentsDepartureTimeRaw_hour"].dt.hour.astype("int8")

    return df

# Cabin Class Encoding
cabin_class_encoding = {
    "coach": 1,
    "premium coach": 2,
    "business": 3,
    "first": 4,
}

def process_segments_CabinClass(df):
    # Split the Cabin class
    df["CabinCode"] = df["segmentsCabinCode"].str.split("\\|\\|")
    
    # apply cabin_class_encoding on the cabin class
    df["CabinCode"] = df["CabinCode"].apply(lambda x: [cabin_class_encoding.get(i, 0) for i in x])

    # get the average of the cabin class
    df["CabinCode"] = df["CabinCode"].apply(lambda x: np.mean(x) if isinstance(x, list) else x)

    return df

# Airline Name Processing

# Airline Encoding 
# 1: Ultra Low Cost, 2: Budget, 3: Regional, 4: Full Service
airline_type_encoding = {
    'JetBlue Airways': 2,
    'Sun Country Airlines': 2,
    'United': 4,
    'Delta': 4,
    'Key Lime Air': 3,
    'Boutique Air': 3,
    'Contour Airlines': 3,
    'Spirit Airlines': 1,
    'American Airlines': 4,
    'Alaska Airlines': 4,
    'Southern Airways Express': 3,
    'Frontier Airlines': 1,
    'Hawaiian Airlines': 4,
    'Cape Air': 3
}

# Airline Category Mapping
def process_segments_AirlineNameScore(df):
    df["AirlineNameScore"] = df["segmentsAirlineName"].str.split("\\|\\|")
    df["AirlineNameScore"] = df["AirlineNameScore"].apply(lambda x: [airline_type_encoding.get(i, 0) for i in x])
    df["AirlineNameScore"]= df["AirlineNameScore"].apply(lambda x: np.max(x) if isinstance(x, list) else x) 
    return df






### Date

In [ ]:
def add_time_difference_columns(df):
    # Calculate the difference in days
    df['date_diff_days'] = (df["flightDate"] - df["searchDate"]).dt.days

    # Calculate sin, cos to train cyclical patterns
    # Day of the week
    df['weekday'] = df["flightDate"].dt.weekday
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
    df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
    
    return df

# Merge into one master
